In [3]:
import numpy as np
import pandas as pd

LL_FINAL = pd.read_csv("/Users/INPUT.csv")
AGG = LL_FINAL.groupby(['ACL_Reporting_Seg'], as_index=False).sum()
ACL_Q_TEMPLATE = pd.read_excel("/Users/Q_FACTOR_INPUT.XLSX")

# Merge two Dataframes on the same columns
LL_QFACTOR = LL_FINAL.merge(ACL_Q_TEMPLATE, how='left', on='ACL_Reporting_Seg')
AGG['LOSSRATE_AGGR']= (AGG['agg_funded_losses']+AGG['agg_unfunded_losses'])/ AGG['UNPAID_BALANCE_AMT']
AGG= AGG.drop(['agg_funded_losses', 'agg_unfunded_losses', 'UNPAID_BALANCE_AMT', 'UnusedCmmtCurrentAmount', 'agg_credit_losses'], axis=1)
AGG

LL_POSTQ = LL_QFACTOR.merge(AGG, how='left', on='ACL_Reporting_Seg')
LL_POSTQ['Q_FACTOR_ADJ_FUNDED'] = LL_POSTQ['agg_funded_losses'] * (LL_POSTQ['LOSSRATE_AGGR'] + LL_POSTQ['Qfactor']/10000) / LL_POSTQ['LOSSRATE_AGGR']
LL_POSTQ['Q_FACTOR_ADJ_UNFUNDED'] = LL_POSTQ['agg_unfunded_losses'] * (LL_POSTQ['LOSSRATE_AGGR'] + LL_POSTQ['Qfactor']/10000) / LL_POSTQ['LOSSRATE_AGGR']
LL_POSTQ['Q_FACTOR_ADJ_AGG'] = LL_POSTQ['Q_FACTOR_ADJ_FUNDED'] + LL_POSTQ['Q_FACTOR_ADJ_UNFUNDED']
LL_POSTQ['funded_qfactor'] = LL_POSTQ['Q_FACTOR_ADJ_FUNDED'] - LL_POSTQ['agg_funded_losses']
LL_POSTQ['unfunded_qfactor'] = LL_POSTQ['Q_FACTOR_ADJ_UNFUNDED'] - LL_POSTQ['agg_unfunded_losses']
LL_POSTQ= LL_POSTQ.drop(['INSTID', 'PD_SEGMENT_ID', 'MATURITY_DT', 'agg_credit_losses', 'agg_funded_losses', 'agg_unfunded_losses', 'Qfactor'], axis=1)

AGG_POSTQ = LL_POSTQ.groupby(['ACL_Reporting_Seg'], as_index=False).sum()
AGG_POSTQ = AGG_POSTQ.drop(['LOSSRATE_AGGR'], axis=1)

LOSSRATE_AGGR = LL_POSTQ.groupby(['ACL_Reporting_Seg'], as_index=False)['LOSSRATE_AGGR'].min()

SEG_ADJQ_SUMMARY = AGG_POSTQ.merge(LOSSRATE_AGGR, how='left', on='ACL_Reporting_Seg')
SEG_ADJQ_SUMMARY['NET_FUNDED_LOSS_RT'] = (SEG_ADJQ_SUMMARY['Q_FACTOR_ADJ_FUNDED']+ SEG_ADJQ_SUMMARY['Q_FACTOR_ADJ_UNFUNDED']) /SEG_ADJQ_SUMMARY['UNPAID_BALANCE_AMT'] 
SEG_ADJQ_SUMMARY['QFACTOR_BPS'] = (SEG_ADJQ_SUMMARY['NET_FUNDED_LOSS_RT'] - SEG_ADJQ_SUMMARY['LOSSRATE_AGGR']) * 10000 

SEG_ADJQ_SUMMARY.to_excel(r'/Users/OUTPUT.xlsx')